## Importing Libraries

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window

**2783. Flight Occupancy and Waitlist Analysis (Medium)**

**Table: Flights**

| Column Name | Type |
|-------------|------|
| flight_id   | int  |
| capacity    | int  |

flight_id is the column with unique values for this table.
Each row of this table contains flight id and its capacity.

**Table: Passengers**

| Column Name  | Type |
|--------------|------|
| passenger_id | int  |
| flight_id    | int  |

passenger_id is the column with unique values for this table.
Each row of this table contains passenger id and flight id.

Passengers book tickets for flights in advance. If a passenger books a ticket for a flight and there are still empty seats available on the flight, the passenger ticket will be confirmed. However, the passenger will be on a waitlist if the flight is already at full capacity.

**Write a solution to report the number of passengers who successfully booked a flight (got a seat) and the number of passengers who are on the waitlist for each flight.**

Return the result table ordered by flight_id in ascending order.

The result format is in the following example.

**Example 1:**

**Input:** 

**Flights table:**
| flight_id | capacity |
|-----------|----------|
| 1         | 2        |
| 2         | 2        |
| 3         | 1        |

**Passengers table:**
| passenger_id | flight_id |
|--------------|-----------|
| 101          | 1         |
| 102          | 1         |
| 103          | 1         |
| 104          | 2         |
| 105          | 2         |
| 106          | 3         |
| 107          | 3         |

**Output:** 
| flight_id | booked_cnt | waitlist_cnt |
|-----------|------------|--------------|
| 1         | 2          | 1            |
| 2         | 2          | 0            |
| 3         | 1          | 1            |

**Explanation:** 
- Flight 1 has a capacity of 2. As there are 3 passengers who have booked tickets, only 2 passengers can get a seat. Therefore, 2 passengers are successfully booked, and 1 passenger is on the waitlist.
- Flight 2 has a capacity of 2. Since there are exactly 2 passengers who booked tickets, everyone can secure a seat. As a result, 2 passengers successfully booked their seats and there are no passengers on the waitlist.
- Flight 3 has a capacity of 1. As there are 2 passengers who have booked tickets, only 1 passenger can get a seat. Therefore, 1 passenger is successfully booked, and 1 passenger is on the waitlist.


In [0]:
flights_data_2783 = [
    (1, 2),
    (2, 2),
    (3, 1)
]

flights_columns_2783 = ["flight_id", "capacity"]
flights_df_2783 = spark.createDataFrame(flights_data_2783, flights_columns_2783)
flights_df_2783.show()

passengers_data_2783 = [
    (101, 1),
    (102, 1),
    (103, 1),
    (104, 2),
    (105, 2),
    (106, 3),
    (107, 3)
]

passengers_columns_2783 = ["passenger_id", "flight_id"]
passengers_df_2783 = spark.createDataFrame(passengers_data_2783, passengers_columns_2783)
passengers_df_2783.show()

+---------+--------+
|flight_id|capacity|
+---------+--------+
|        1|       2|
|        2|       2|
|        3|       1|
+---------+--------+

+------------+---------+
|passenger_id|flight_id|
+------------+---------+
|         101|        1|
|         102|        1|
|         103|        1|
|         104|        2|
|         105|        2|
|         106|        3|
|         107|        3|
+------------+---------+



In [0]:
passengers_df_2783 = passengers_df_2783\
                            .groupBy("flight_id")\
                                .agg(count("*").alias("total_passengers"))


In [0]:
joined_df_2783 = flights_df_2783\
                        .join(
                            passengers_df_2783, on="flight_id", how="left"
                            ).fillna(0)

In [0]:
joined_df_2783\
    .withColumn(
        "booked_cnt",
        least(col("capacity"), col("total_passengers")))\
    .withColumn(
        "waitlist_cnt",
        greatest(col("total_passengers") - col("capacity"), lit(0)))\
            .select("flight_id", "booked_cnt", "waitlist_cnt").orderBy("flight_id").display()

flight_id,booked_cnt,waitlist_cnt
1,2,1
2,2,0
3,1,1
